In [93]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
import dtcv
from shapely import wkt
from shapely.geometry import Polygon, LineString, Point
import json 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [94]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# San Diego Downtown Homless Computer Vision Package
`sandiegodata.org-downtown_cv-4` Last Update: 2019-09-13T00:17:29

_Files and code for analyzing San Diego downtown homelessness  data with computer vision_


This dataset collects records related to a conversion of 5 years of paper maps
that record positions of homeless sleepers in downtown San Diego. The San Diego
Regional Data Library is [converting these paper maps to a digital
form](http://downtown-homelessness.sandiegodata.org/) with a manual process that
uses an image annotation tool, and theses annotations can be used to train
computer vision algorithms to georeference maps and recognize handwritten marks.

These datasets link to map urls and annotations, for three kinds of annotations:

* Ground Control Points, which identify the map image locations for known intersections, linking image coordinates ( in pixels ) to geographic coordinates.
* Image locations of handwritten marks and the number written in the mark.
* File annotations, for other handwritten notes such as the temperature and presence of rain. 

For more discussion about the GCP and handwritten marks, and the tasks in volved
in developing computer vision algorithms for these data, [see our recent blog
post on the subject.
](https://www.sandiegodata.org/2019/09/computer-vision-for-greater-good/)

For some examples of using OpenCV to extract and match templates, to georeference maps, see the [Templates and Clustering Jupyter Notebook](https://nbviewer.jupyter.org/github/sandiegodata-projects/homelessness/blob/master/datasets/sandiegodata.org-downtown_cv/notebooks/Template%20Matching%20Clusters.ipynb)


## Developer notes

After anotation JSON files are copied into S#, the list of S# urls must be
updated. To refresh the list of urls run

    $  bin/update_s3.sh <s3-profile>


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [gcp](notebooks/rundf.ipynb#gcp)**. Ground control points
* ** [intersection_regions](notebooks/rundf.ipynb#intersection_regions)**. Polygon transformations for each the intersections of each map
* ** [intersections](notebooks/rundf.ipynb#intersections)**. List of intersections.
* ** [file_annotations](notebooks/rundf.ipynb#fa)**. File annotations on count files
* ** [counts](notebooks/rundf.ipynb#count)**. Annotation position, types and counts of handwritten marks

In [95]:
gcp = dtcv.gcp_df()
gcp.head()

,image_url,x,y,width,height,intersection
0,http://ds.civicknowledge.org/downtownsandiego....,248,848,29,58,Ketner_Broadway
1,http://ds.civicknowledge.org/downtownsandiego....,573,851,25,61,State_Broadway
2,http://ds.civicknowledge.org/downtownsandiego....,575,244,31,31,State_A
3,http://ds.civicknowledge.org/downtownsandiego....,251,240,22,34,Ketner_A
4,http://ds.civicknowledge.org/downtownsandiego....,807,1226,41,41,Ketner_Broadway


In [96]:
def uninvert_point(p):
    x,y = p

    y = -y + 2000
    return (x,y)

def uninvert_poly(poly):

    return Polygon( uninvert_point(p) for p in poly_to_array(poly))

def poly_to_array(poly):

    try:
        
        return  np.array(poly.exterior.coords)[:-1]
    except AttributeError:
        return poly
    
df = dtcv.gcp_transform_df().rename(columns={'source':'source_inv'})

df['source'] = df.source_inv.apply(uninvert_poly)
df['source_area'] = df.source.apply(lambda v: v.area)

def bound_dims(v):
    (minx, miny, maxx, maxy) = v.bounds
    return round(maxx-minx,-1), round(maxy-miny,-1)

df['source_shape'] = df.source.apply(bound_dims)
df['source_shape_x'] = df.source_shape.apply(lambda v: v[0])
df['source_shape_y'] = df.source_shape.apply(lambda v: v[1])

from pathlib import Path 

def agg_inter(g):
    return ','.join(sorted(g.unique()))

t =gcp.rename(columns={'image_url':'url'}).groupby('url').agg({'intersection':agg_inter}).reset_index()\
    .rename(columns={'intersection':'intersections_id'})
df = df.merge(t, on='url')

df['year'] = df.url.apply(lambda v: int(Path(v).stem[:4]))
df['month'] = df.url.apply(lambda v: int(Path(v).stem[4:6]))


# some extra variant information
def map_names(r):

    ym = r.year*100 + r.month
    
    if  ym < 201600:
        return r.neighborhood+'_14'
    elif 201600 <= ym < 201704:
        return r.neighborhood+'_16'
    elif 201704 <= ym:
        return r.neighborhood+'_17'
    else:
        # Should not get here. 
        print(ym, r.year, r.month)
        return r.neighborhood+'_18'
        

df['map_name'] =  df.apply(map_names, axis=1)
   
# Different maps variants, based on the intersections
intersection_group_map = {
    'Ketner_A,Ketner_Broadway,State_A,State_Broadway':   'a', #'columbia',
    '11th_B,11th_Broadway,Front_B,Front_Broadway':       'a', #'core_columbia',
    '1st_Ash,1st_Cedar,9th_Ash,9th_Cedar':               'a', #'cortez',
    '16th_E,16th_Imperial,7th_E,7th_Imperial':           'a', #'east_village_a',
    '16th_Imperial,16th_Market,7th_Imperial,7th_Market': 'b', #'east_village_b',
    '16th_E,16th_Market,7th_E,7th_Market':               'c', #'east_village_c',
    '4th_Broadway,4th_K,6th_Broadway,6th_K':             'a', #'gaslamp',
    '3rd_G,3rd_K,Ketner_G,Ketner_Market':                'a'} #'marina'}

df['intersection_group'] = t.intersections_id.apply(lambda v: intersection_group_map[v] )


intersection_regions = df[['url', 'neighborhood', 'year', 'month','intersections_id',  'intersection_group', 'map_name',
         'source_inv', 'source', 'source_area', 'source_shape', 'source_shape_x', 'source_shape_y',
         'dest', 'matrix' ]]

intersection_regions.head().T


,0,1,2,3,4
url,http://ds.civicknowledge.org/downtownsandiego....,http://ds.civicknowledge.org/downtownsandiego....,http://ds.civicknowledge.org/downtownsandiego....,http://ds.civicknowledge.org/downtownsandiego....,http://ds.civicknowledge.org/downtownsandiego....
neighborhood,columbia,columbia,columbia,columbia,columbia
year,2015,2016,2016,2016,2016
month,9,1,2,3,4
intersections_id,"Ketner_A,Ketner_Broadway,State_A,State_Broadway","Ketner_A,Ketner_Broadway,State_A,State_Broadway","Ketner_A,Ketner_Broadway,State_A,State_Broadway","Ketner_A,Ketner_Broadway,State_A,State_Broadway","Ketner_A,Ketner_Broadway,State_A,State_Broadway"
intersection_group,a,a,a,a,a
map_name,columbia_14,columbia_16,columbia_16,columbia_16,columbia_16
source_inv,"POLYGON ((262 1743, 262.5 1123, 585.5 1118.5, ...","POLYGON ((832.5 1161, 827.5 753.5, 1541.5 750....","POLYGON ((605 1426.5, 602 1086, 1188 1086, 118...","POLYGON ((844.5 1227.5, 848.5 777.5, 1626.5 77...","POLYGON ((627 1423, 626 1097.5, 1186.5 1095.5,..."
source,"POLYGON ((262 257, 262.5 877, 585.5 881.5, 590...","POLYGON ((832.5 839, 827.5 1246.5, 1541.5 1249...","POLYGON ((605 573.5, 602 914, 1188 914, 1185 5...","POLYGON ((844.5 772.5, 848.5 1222.5, 1626.5 12...","POLYGON ((627 577, 626 902.5, 1186.5 904.5, 11..."
source_area,202299,291446,197346,352782,182724


In [5]:
intersections = dtcv.intersections_df()
intersections.head().T

,0,1,2,3,4
geometry,POINT (6279769 1841430.99999986),POINT (6279780 1842558.99999987),POINT (6278946 1841436.99999983),POINT (6278953.17300012 1842563.14399985),POINT (6280315 1841426.99999984)
neighborhood,columbia,columbia,columbia,columbia,core_columbia
intersection,State_Broadway,State_A,Ketner_Broadway,Ketner_A,Front_Broadway


In [6]:
intersection_regions.columns

Index(['url', 'neighborhood', 'year', 'month', 'intersections_id',
       'intersection_group', 'map_name', 'source_inv', 'source', 'source_area',
       'source_shape', 'source_shape_x', 'source_shape_y', 'dest', 'matrix'],
      dtype='object')

In [92]:

fa = dtcv.file_df()

n_map = {
    'Columbia': 'columbia',
    'east_vllage': 'east_village',
    'East Village':  'east_village',
    'East Village N': 'east_village',
    'east villiage':  'east_village',
    'Gaslamp': 'gaslamp',
    'Marina': 'marina', 
}

fa = fa.replace('2017-03-30\n','2017-03-30') # Error in one of the dates
fa['neighborhood'] = fa.neighborhood.replace(n_map)
fa['rain'] = fa.rain.fillna('')
fa['temp'] = pd.to_numeric(fa.temp, errors='ignore')

from pathlib import Path
fa['url_year'] = fa.image_url.apply(lambda v: int(Path(v.strip()).stem[0:4]))
fa['url_month'] = fa.image_url.apply(lambda v: int(Path(v.strip()).stem[4:6]))
fa['date'] = pd.to_datetime(fa.apply(lambda r: r.date if r.date else f'{r.url_year}-{r.url_month:02d}-01' , axis=1))
fa['url_neighborhood']  = fa.image_url.apply(lambda v: v.strip().split('/')[-2])
fa['neighborhood']  = fa.apply(lambda r: r.neighborhood if r.neighborhood else r.url_neighborhood, axis =1 )
 

fa = fa[['image_url', 'url_year', 'url_month', 'date', 'neighborhood', 'url_neighborhood', 'total_count', 'temp', 'rain',]]
fa.head()

,image_url,url_year,url_month,date,neighborhood,url_neighborhood,total_count,temp,rain
0,http://ds.civicknowledge.org/downtownsandiego....,2015,9,2015-09-25,columbia,columbia,41,,clear
1,http://ds.civicknowledge.org/downtownsandiego....,2016,1,2016-01-29,columbia,columbia,50,None,clear
2,http://ds.civicknowledge.org/downtownsandiego....,2016,2,2016-02-25,columbia,columbia,45,None,clear
3,http://ds.civicknowledge.org/downtownsandiego....,2016,3,2016-03-01,columbia,columbia,65,None,clear
4,http://ds.civicknowledge.org/downtownsandiego....,2016,4,2016-04-01,columbia,columbia,30,None,clear


In [82]:
count = dtcv.count_df()


,image_url,cx,cy,r,type,count
0,http://ds.civicknowledge.org/downtownsandiego....,654,100,40.000,Individual,2
1,http://ds.civicknowledge.org/downtownsandiego....,658,188,34.000,Individual,6
2,http://ds.civicknowledge.org/downtownsandiego....,656,319,37.000,Individual,6
3,http://ds.civicknowledge.org/downtownsandiego....,630,411,30.900,Individual,7
4,http://ds.civicknowledge.org/downtownsandiego....,105,620,57.096,Individual,1


0